In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
from multiprocessing import Pool
import datetime
from typing import List, Optional

from examples.agent_market.agents import (
    MM_WALLET,
    TERMINATE_WALLET,
    TRADER_1_WALLET,
    TRADER_2_WALLET,
    TRADER_3_WALLET,
    TRADER_4_WALLET,
    MarketMaker,
    MarketOrderTraders,
    random_walk_price,
    StateAgentWithWallet
)
from vega_sim.environment import MarketEnvironmentWithState
from vega_sim.null_service import VegaServiceNull

trader_wallets = [
    TRADER_1_WALLET,
    TRADER_2_WALLET,
    TRADER_3_WALLET,
    TRADER_4_WALLET
]

In [ ]:
def build_env(
    num_steps: int = 60,
    block_size: int = 1,
    order_arrival_rate: float = 2,
    vega: Optional[VegaServiceNull] = None,
    state_extraction_fn = None
):
    market_maker = MarketMaker(
        wallet_name=MM_WALLET.name,
        wallet_pass=MM_WALLET.passphrase,
        terminate_wallet_name=TERMINATE_WALLET.name,
        terminate_wallet_pass=TERMINATE_WALLET.passphrase,
        price_process=random_walk_price(
            terminal_time_seconds=num_steps + 10, sigma=0.001, initial_price=20, min_price=2
        ),
        spread=0.2,
    )
    mo_traders = [StateAgentWithWallet(w.name, w.passphrase) for w in trader_wallets]
    mo_trader_agent = MarketOrderTraders(
        mo_traders[:1],
        buy_order_arrival_rate=order_arrival_rate,
    )
    env = MarketEnvironmentWithState(
        agents=[market_maker, mo_trader_agent],
        n_steps=num_steps,
        transactions_per_block=block_size,
        vega_service=vega,
        state_extraction_fn=state_extraction_fn
    )
    return env

In [ ]:
with VegaServiceNull(warn_on_raw_data_access=False, use_full_vega_wallet=False, run_with_console=False, start_order_feed=True) as vega:
    import time
    time.sleep(2)
    print(vega.wallet_url)
    print(vega.data_node_graphql_port)
    env = build_env(
        **{
            "num_steps": int(1 * 60),
            "block_size": 10,
            "vega": vega,
            "state_extraction_fn": lambda veg, agents: (veg.best_prices(agents[0].market_id), veg.party_account(agents[1].market_agents[0].wallet_name, agents[0].asset_id, agents[0].market_id))
        },
    )
    results = env.run(run_with_console=False, pause_at_completion=False)

In [ ]:
import numpy as np

In [ ]:
res = np.array([r[0] for r in results])
bids = res[:, 0]
asks = res[:, 1]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(bids)
plt.plot(asks)
plt.show()